In [15]:
# Establishing the Molecule of choice
from qiskit_nature.second_q.drivers import PySCFDriver
driver = PySCFDriver(atom="H 0 0 0; H 0 0 0.76", basis="sto-3g")

# Loading the molecular configuration [Hamiltonian]
problem = driver.run()

In [16]:
from qiskit_nature.second_q.mappers import JordanWignerMapper, BravyiKitaevMapper
#mapper = BravyiKitaevMapper()

# Mapping the Molecular Hamiltonian to a Qubit Hamiltonian
mapper = JordanWignerMapper()

In [17]:
# Creating an Ansatz which is an assumption of the quantum state. The UCCSD (Unitary Coupled Cluster Singles and Doubles) is the type of Ansatz used for VQE
# Hartree Fock sets the initial state (The parameters like number of orbitals, number of electron, etc)

from qiskit_nature.second_q.circuit.library import UCCSD, HartreeFock
ansatz = UCCSD(
    problem.num_spatial_orbitals,
    problem.num_particles,
    mapper,
    initial_state=HartreeFock(
        problem.num_spatial_orbitals,
        problem.num_particles,
        mapper,
    ),
)
num_qubits_allocated = ansatz.num_qubits
print("Number of qubits allocated:", num_qubits_allocated)

Number of qubits allocated: 4


In [18]:
#Initializing the VQE with an estimator to compute the energy, with a SPSA optimizer for the Ansatz
import numpy as np
from qiskit.algorithms.optimizers import SPSA  #Simultaneous Perturbation Stochastic Approximation
from qiskit.algorithms.minimum_eigensolvers import VQE
from qiskit.primitives import Estimator
vqe = VQE(Estimator(), ansatz, SPSA())

In [19]:
# Initializing all parameters to zeros
vqe.initial_point = np.zeros(ansatz.num_parameters)

In [7]:
from qiskit_nature.second_q.algorithms import GroundStateEigensolver
solver = GroundStateEigensolver(mapper, vqe)
result = solver.solve(problem)

print(f"Total ground state energy = {result.total_energies[0]:.4f}")

Total ground state energy = -1.1366


Running the VQE on a Backend

In [20]:
from azure.quantum.qiskit import AzureQuantumProvider
provider = AzureQuantumProvider(
            resource_id = "",
            location = "East US")

In [21]:
print("This workspace's targets:")
for backend in provider.backends():
    print("- " + backend.name())

This workspace's targets:
- ionq.simulator
- ionq.simulator
- ionq.simulator
- ionq.qpu
- ionq.qpu
- ionq.qpu
- ionq.qpu.aria-1
- ionq.qpu.aria-1
- ionq.qpu.aria-1
- quantinuum.hqs-lt-s1-apival
- quantinuum.hqs-lt-s1-apival
- quantinuum.sim.h1-1sc
- quantinuum.sim.h1-1sc
- quantinuum.hqs-lt-s2-apival
- quantinuum.hqs-lt-s2-apival
- quantinuum.sim.h1-2sc
- quantinuum.sim.h1-2sc
- quantinuum.hqs-lt-s1-sim
- quantinuum.hqs-lt-s1-sim
- quantinuum.sim.h1-1e
- quantinuum.sim.h1-1e
- quantinuum.hqs-lt-s2-sim
- quantinuum.hqs-lt-s2-sim
- quantinuum.sim.h1-2e
- quantinuum.sim.h1-2e
- quantinuum.hqs-lt-s1
- quantinuum.hqs-lt-s1
- quantinuum.qpu.h1-1
- quantinuum.qpu.h1-1
- quantinuum.hqs-lt-s2
- quantinuum.hqs-lt-s2
- quantinuum.qpu.h1-2
- quantinuum.qpu.h1-2
- microsoft.estimator


In [22]:
ionq_sim = provider.get_backend('ionq.qpu.aria-1')
backend_to_use = ionq_sim

In [23]:
from qiskit_nature.second_q.algorithms import GroundStateEigensolver

# The GroundStateEigensolver creates an instance of the class and initializes it with the vqe and mapper
solver = GroundStateEigensolver(mapper, vqe)

# Creating a session to be submitted to the QPU
with backend_to_use.open_session(name="VQE H2") as session:
    backend_props = backend_to_use.properties()
   
    result = solver.solve(problem)

num_qubits_ionq = backend_to_use.configuration().n_qubits

print("Number of qubits used in IonQ:", num_qubits_ionq)

Number of qubits used in IonQ: 23


In [24]:
# Printing the result
print("AzureQuantum " + backend_to_use.name() + " result:\n")
print(result.groundenergy)

AzureQuantum ionq.qpu.aria-1 result:

-1.8328524309582364
